In [4]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

In [2]:
data_path = r'C:\Users\91767\OneDrive\Documents\fdata_w.csv'
data = pd.read_csv(data_path)

In [5]:
X = data
kmeans = KMeans(n_clusters=2, random_state=42)
y = kmeans.fit_predict(X)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [8]:
dt_model = DecisionTreeClassifier(random_state=42)
dt_model.fit(X_train, y_train)
dt_predictions = dt_model.predict(X_test)

In [9]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
rf_predictions = rf_model.predict(X_test)

In [10]:
gb_model = GradientBoostingClassifier(n_estimators=100, random_state=42)
gb_model.fit(X_train, y_train)
gb_predictions = gb_model.predict(X_test)

In [11]:
svm_model = SVC(probability=True, random_state=42)
svm_model.fit(X_train, y_train)
svm_predictions = svm_model.predict(X_test)

In [12]:
def create_ann_model():
    model = Sequential()
    model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [13]:
ann_model = KerasClassifier(build_fn=create_ann_model, epochs=50, batch_size=64, verbose=0)
ann_model.fit(X_train, y_train)
ann_predictions = ann_model.predict(X_test)

C:\Users\91767\AppData\Local\Temp\ipykernel_14512\868258244.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  ann_model = KerasClassifier(build_fn=create_ann_model, epochs=50, batch_size=64, verbose=0)


186/186 [==============================] - 1s 3ms/step


In [14]:
X_train_rnn = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test_rnn = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))
y_train_categorical = to_categorical(y_train)
y_test_categorical = to_categorical(y_test)

In [15]:
rnn_model = Sequential()
rnn_model.add(LSTM(50, input_shape=(1, X_train.shape[1]), return_sequences=True))
rnn_model.add(Dropout(0.2))
rnn_model.add(LSTM(50, return_sequences=False))
rnn_model.add(Dropout(0.2))
rnn_model.add(Dense(y_train_categorical.shape[1], activation='softmax'))

rnn_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
rnn_model.fit(X_train_rnn, y_train_categorical, epochs=50, batch_size=64, validation_data=(X_test_rnn, y_test_categorical))
rnn_predictions = np.argmax(rnn_model.predict(X_test_rnn), axis=1)

Epoch 1/50
370/370 [==============================] - 15s 16ms/step - loss: 0.1653 - accuracy: 0.9564 - val_loss: 0.0383 - val_accuracy: 0.9872
Epoch 2/50
370/370 [==============================] - 3s 9ms/step - loss: 0.0280 - accuracy: 0.9898 - val_loss: 0.0205 - val_accuracy: 0.9922
Epoch 3/50
370/370 [==============================] - 3s 9ms/step - loss: 0.0187 - accuracy: 0.9923 - val_loss: 0.0121 - val_accuracy: 0.9965
Epoch 4/50
370/370 [==============================] - 3s 9ms/step - loss: 0.0136 - accuracy: 0.9951 - val_loss: 0.0130 - val_accuracy: 0.9943
Epoch 5/50
370/370 [==============================] - 3s 9ms/step - loss: 0.0122 - accuracy: 0.9954 - val_loss: 0.0084 - val_accuracy: 0.9971
Epoch 6/50
370/370 [==============================] - 3s 9ms/step - loss: 0.0108 - accuracy: 0.9956 - val_loss: 0.0086 - val_accuracy: 0.9965
Epoch 7/50
370/370 [==============================] - 3s 9ms/step - loss: 0.0099 - accuracy: 0.9961 - val_loss: 0.0090 - val_accuracy: 0.9958
Epoc

In [16]:
stacked_predictions = np.column_stack((dt_predictions, rf_predictions, gb_predictions, svm_predictions, ann_predictions, rnn_predictions))

In [17]:
meta_model = LogisticRegression()
meta_model.fit(stacked_predictions, y_test)
meta_predictions = meta_model.predict(stacked_predictions)

In [18]:
ensemble_accuracy = accuracy_score(y_test, meta_predictions)
ensemble_classification_report = classification_report(y_test, meta_predictions)

In [19]:
print(f'Ensemble Accuracy: {ensemble_accuracy}')
print(f'Ensemble Classification Report:\n{ensemble_classification_report}')

Ensemble Accuracy: 0.9996622192197264
Ensemble Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4646
           1       1.00      1.00      1.00      1275

    accuracy                           1.00      5921
   macro avg       1.00      1.00      1.00      5921
weighted avg       1.00      1.00      1.00      5921

